# Comparing Normalized & Conf Models
Via XGB.train kunnen wij aparte confidence berekenen
via xgbclassifier.fit niet

In [1]:
# importeren packages
import pandas as pd
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score 
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib import pyplot
from sklearn.preprocessing import RobustScaler

In [2]:
# inladen bestand met scores als sc
sc = pd.read_excel('P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# alleen selecteren kolommen voor scores en oordelen
sc = sc[['buurtcode','EE','TK']]

# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv('P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\buurten.csv', encoding='utf-16')

C:\Users\sikkesr\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = pd.merge(sc,buurtendata, on='buurtcode')

In [4]:
# feature_cols_full = ['inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL16', 'KL18', 'VKL1618',
#                'RLBRMTR18', 'RLBWON18', 'RLBBEV18', 'RLBVRZ18', 'RLBVEI18', 'RLBFYS18', 
#                'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
#                'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
#                'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
#                'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 
#                'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2', 
#                'woxGem', 'woxdelta1jrGem', 'woxdelta5jrGem', 'woxwwGem', 'woxm2Gem', 'woxdelta5jrWGabs', 'woxdelta5jrWGrel', 
#                'woxm2WG','banen'] 

# feature_cols_full = ['AantalCafe3', 'ar60', 'woxdelta5jr', 'woxm2', 'woxww',
#        'woxdelta5jrGem', 'wox', 'RLBBEV18', 'ov60',
#        'RLBVRZ16', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
#        'woxdelta5jrWGabs', 'woxdelta1jrGem', 'fiets', 'AantalPodiumk10',
#        'banen', 'wv2000-2010', 'AantalBios5']

feature_cols_full = ['woxdelta5jrGem', 'AantalCafe3', 'woxdelta5jr', 'ar60', 'wox',
       'woxdelta1jrGem', 'AantalBios5', 'woxm2WG']

X = df[feature_cols_full]

# Same for all models
replace_dict = {'a': 0, 'b': 1, 'c':2}
y = df['TK'].apply(lambda x: replace_dict[x])

In [5]:
# Creating a scaled feature set
scaler = RobustScaler()
X_scaled = scaler.fit_transform(df[feature_cols_full])
X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = feature_cols_full

In [8]:
#X_scaled.columns

In [6]:
# Same for all models
# replace_dict = {'a': 0, 'b': 1, 'c':2}
# y = df['EE'].apply(lambda x: replace_dict[x])

## Testing
#pd.DataFrame({'original': df['EE'], 'new': y}).to_csv('testing.csv')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X_scaled, y, test_size=0.25, random_state=2)

In [10]:
# Test if split is the same indices are taken
print(sum(y_train_scaled.index.values == y_train.index.values)/len(y_train))
print(sum(X_train_scaled.index.values == X_train.index.values)/len(y_train))

1.0
1.0


In [11]:
# check if < in column names, this is not allowed
#[x for x in X_train.columns if '<' in x]

In [10]:
# For the confidence model we factorize the output
# replace_dict = {'a': 0, 'b': 1, 'c':2}

# y_train = y_train['EE'].apply(lambda x: replace_dict[x])
# y_test = y_test['EE'].apply(lambda x: replace_dict[x])

# y_train_scaled = y_train_scaled['EE'].apply(lambda x: replace_dict[x])
# y_test_scaled = y_test_scaled['EE'].apply(lambda x: replace_dict[x])

# Testing

## Creating models with confidence for both datasets; standard & scaled
### Conf model with standard data

In [12]:
params= {'base_score':0.5, 'booster':'gbtree', 'colsample_bylevel':1,
              'colsample_bynode':1, 'colsample_bytree':0.7, 'gamma':0.2,
              'learning_rate':0.15, 'max_delta_step':0, 'max_depth':7,
              'min_child_weight':3, 'missing':None, 'n_estimators':250, 'n_jobs':1,
              'nthread':None, 'objective':'multi:softprob', 'random_state':0,
              'reg_alpha':0, 'reg_lambda':1, 'scale_pos_weight':1, 'seed':2,
              'silent':None, 'subsample':0.8, 'verbosity':3, 'num_class' : 3}

In [13]:
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test =  xgboost.DMatrix(X_test, label=y_test)
model = xgboost.train(params, dtrain = xgb_train)

test_preds = [x.argmax() for x in model.predict(xgb_test)]

for x,y,z in zip(y_test, test_preds, [x.max() for x in model.predict(xgb_test)]):
    print(f"Truth: {x}, pred:{y}, conf {z}")

[14:24:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:24:12] DEBUG: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\gbm\gbtree.cc:146: Using tree method: 2
[14:24:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 18 extra nodes, 2 pruned nodes, max_depth=4
[14:24:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[14:24:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 12 

In [15]:
#model.predict(xgb_test)

In [41]:
model.save_config()

'{"learner":{"generic_param":{"enable_experimental_json_serialization":"0","gpu_id":"-1","gpu_page_size":"0","n_gpus":"0","n_jobs":"1","nthread":"1","random_state":"2","seed":"2","seed_per_iteration":"0","validate_features":"0","validate_parameters":"1"},"gradient_booster":{"gbtree_train_param":{"num_parallel_tree":"1","predictor":"auto","process_type":"default","tree_method":"exact","updater":"grow_colmaker,prune","updater_seq":"grow_colmaker,prune"},"name":"gbtree","specified_updater":false,"updater":{"grow_colmaker":{"colmaker_train_param":{"opt_dense_col":"1"},"train_param":{"alpha":"0","cache_opt":"1","colsample_bylevel":"1","colsample_bynode":"1","colsample_bytree":"0.699999988","default_direction":"learn","enable_feature_grouping":"0","eta":"0.150000006","gamma":"0.200000003","grow_policy":"depthwise","interaction_constraints":"","lambda":"1","learning_rate":"0.150000006","max_bin":"256","max_conflict_rate":"0","max_delta_step":"0","max_depth":"7","max_leaves":"0","max_search_gr

In [16]:
#train_accuracy = accuracy_score(y_train,test_preds)
test_accuracy = accuracy_score(y_test,test_preds)
cm = confusion_matrix(y_test, test_preds)

In [17]:
test_accuracy

0.8823529411764706

In [18]:
model.get_score(importance_type='gain')

{'AantalCafe3': 4.013762429719297,
 'woxdelta5jr': 3.2016668324042556,
 'woxm2WG': 2.5807303785640823,
 'wox': 8.172626356470587,
 'woxdelta5jrGem': 6.350791487375,
 'ar60': 4.80833654783784,
 'AantalBios5': 4.18090753156,
 'woxdelta1jrGem': 1.6339578223333333}

In [19]:
standard_conf_preds = test_preds

### Conf model with scaled data

In [20]:
X_train[:1]

,woxdelta5jrGem,AantalCafe3,woxdelta5jr,ar60,wox,woxdelta1jrGem,AantalBios5,woxm2WG
300,43.9,9.5,43.5,3.0,283.0,7.6,0.0,0.871795


In [21]:
X_train_scaled[:1]

,woxdelta5jrGem,AantalCafe3,woxdelta5jr,ar60,wox,woxdelta1jrGem,AantalBios5,woxm2WG
300,0.0,-0.011662,-0.172414,-0.509936,-0.244344,0.186441,-0.5,-0.631676


In [22]:
y_train_scaled[:1]

300    2
Name: TK, dtype: int64

In [23]:
y_train[:1]

300    2
Name: TK, dtype: int64

In [24]:
xgb_train_scaled = xgboost.DMatrix(X_train_scaled, label=y_train_scaled)
xgb_test_scaled =  xgboost.DMatrix(X_test_scaled, label=y_test_scaled)
model = xgboost.train(params, dtrain = xgb_train_scaled)

test_preds = [x.argmax() for x in model.predict(xgb_test_scaled)]

for x,y,z in zip(y_test_scaled, test_preds, [x.max() for x in model.predict(xgb_test_scaled)]):
    print(f"Truth: {x}, pred:{y}, conf {z}")

[14:27:12] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:27:12] DEBUG: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\gbm\gbtree.cc:146: Using tree method: 2
[14:27:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 18 extra nodes, 2 pruned nodes, max_depth=4
[14:27:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 18 extra nodes, 0 pruned nodes, max_depth=5
[14:27:12] INFO: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\tree\updater_prune.cc:89: tree pruning end, 12 

In [ ]:
# confidence verifieren. 

In [25]:
normalized_conf_preds = test_preds

# Standard Model standard data

In [26]:
# maak object met geoptimaliseerde hyperparameters
standard_model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob',softmax_multiclass_param= 3,random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
standard_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              softmax_multiclass_param=3, subsample=0.8, tree_method=None,
              validate_parameters=False, ...)

In [25]:
#standard_model.get_xgb_params

In [27]:
y_pred = standard_model.predict(X_test)
y_train_pred = standard_model.predict(X_train)

In [28]:
train_accuracy = accuracy_score(y_train,y_train_pred)
test_accuracy = accuracy_score(y_test,y_pred)
cm = confusion_matrix(y_test, y_pred)
test_accuracy

0.9058823529411765

In [29]:
standard_standard_preds = y_pred

In [30]:
standard_standard_preds

array([1, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 2, 2, 1, 2, 2, 0, 0, 0, 1, 2, 0, 0, 1, 0, 0, 2, 2, 0, 1, 0,
       2, 0, 0, 1, 0, 0, 2, 1, 2, 1, 1, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2,
       0, 1, 0, 2, 2, 1, 2, 0, 1, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2],
      dtype=int64)

In [31]:
y_train

300    2
99     0
257    2
74     2
232    2
      ..
299    2
22     0
72     2
15     1
168    0
Name: TK, Length: 253, dtype: int64

In [31]:
# fi = pd.DataFrame(standard_model.feature_importances_,X_train.columns)
# fi.columns = ['feature_score']
# fi.sort_values('feature_score', ascending=False, inplace=True)
# fi['feature'] = fi.index
# fi

## Standard Model Normalized Data

In [32]:
standard_model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

In [33]:
standard_model.fit(X_train_scaled, y_train_scaled)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [34]:
standard_model.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)>

In [35]:
y_pred = standard_model.predict(X_test_scaled)
y_train_pred = standard_model.predict(X_train_scaled)

In [36]:
train_accuracy = accuracy_score(y_train_scaled,y_train_pred)
test_accuracy = accuracy_score(y_test_scaled,y_pred)
cm = confusion_matrix(y_test, y_pred)
test_accuracy

0.9058823529411765

In [37]:
cm

array([[38,  2,  0],
       [ 3, 11,  2],
       [ 1,  0, 28]], dtype=int64)

In [38]:
standard_normalized_preds = y_pred

# Comparing Results

In [39]:
results = pd.DataFrame({
    'conf_standard': standard_conf_preds,
    'conf_normalized': normalized_conf_preds,
    'standard_standard': standard_standard_preds,
    'standard_normalized': standard_normalized_preds
})
#     standard_normalized_preds
# standard_standard_preds
# normalized_conf_preds
# standard_conf_preds
results.head()

,conf_standard,conf_normalized,standard_standard,standard_normalized
0,1,1,1,1
1,0,0,0,0
2,2,2,2,2
3,2,2,2,2
4,0,0,0,0


In [39]:
#pd.Series(results.sum(axis=1)%4)

0     2
1     0
2     0
3     0
4     0
     ..
80    0
81    0
82    0
83    0
84    0
Length: 85, dtype: int64

In [40]:
#pd.Series(results.sum(axis=1)%4).sum()

In [41]:
results.corr()

,conf_standard,conf_normalized,standard_standard,standard_normalized
conf_standard,1.000000,1.000000,0.942874,0.942874
conf_normalized,1.000000,1.000000,0.942874,0.942874
standard_standard,0.942874,0.942874,1.000000,1.000000
standard_normalized,0.942874,0.942874,1.000000,1.000000


# Zijn dus een paar verschillen

In [44]:
standard_model.save_model('0001.model') 

In [47]:
bst = xgboost.Booster(params)  # init model
bst.load_model('0001.model')  # load data

[14:55:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators, scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [48]:
bst = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)
bst.load_model('0001.model') 

[14:56:16] ======== Monitor: Learner ========
[14:56:16] Configure: 0s, 1 calls @ 0us

[14:56:16] ======== Monitor: GBTree ========


In [49]:
bst

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=250, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)